## <span style="color:orange">Q1.</span>
<b>우리는 IMDB 데이터셋에 Embedding층을 사용해보았습니다.  
<span style="color:red">Embedding층은 사전 학습된 가중치를 불러와 사용할 수 있습니다.</span>

여러 차례 언급했듯이, 사전 학습된 가중치를 사용한다는 것은 모델 성능에 큰 향상을 가져올지도 모릅니다.  
사전 학습된 가중치를 불러온 Embedding층을 활용하여 문제를 해결해보기를 바랍니다.  
밑의 힌트에서 언급한 Glove 또는 Word2Vec 중 하나를 선택하여 진행하세요.</b>
<hr>

## Glove 사용하기
+ (저장소에선 파일을 제공하지 않습니다, 직접 다운받아주세요.) Glove 다운로드
+ 링크는 책에서 제공하고 있습니다.  
<br>
+ 압축을 풀면 glove.6B.100d.txt파일이 보입니다.
+ 100차원을 가지는 임베딩 가중치를 포함하고 있습니다.

#### glove.txt에 존재하는 단어별 가중치를 옮겨담기

In [ ]:
import os
import numpy as np

# glove.6B.100d.txt 파일이 존재하는 경로를 입력해주세요.
glove_dir = 'your_glove_file_path'

embedding_dict = {}

f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding = 'utf-8')

# 가장 첫 번째 값은 'the', ','와 같은 단어이고,
# 1:101은 해당 단어의 가중치를 나타냅니다.
for line in f:
    values = line.split()
    word = values[0] # 단어
    weights = np.array(values[1:], dtype = 'float32') # 가중치
    embedding_dict[word] = weights
    
f.close()

In [ ]:
print(embedding_dict) # 형태를 확인해보세요.

#### imdb 데이터셋에서 사용하는 단어의 가중치만 옮겨담기

In [ ]:
from tensorflow.keras.datasets import imdb

num_words = 10000
embedding_dim = 100

# 케라스 데이터셋에서 word index를 제공해주고 있기 때문에,
# 여기서는 word index를 만드는 작업을 생략하겠습니다.
words_index = imdb.get_word_index()

# embedding_matrix에 가중치를 옮겨 담습니다.
embedding_matrix = np.zeros((num_words, embedding_dim))

# word: 단어, idx는 단어의 인덱스입니다.
for word, idx in words_index.items():
    if(idx < num_words): # 10,000번째 단어까지만 사용합니다.
        word_weight = embedding_dict.get(word)
        if word_weight is not None:
            embedding_matrix[idx] = word_weight

#### imdb 데이터셋 준비

In [ ]:
# imdb 데이터셋을 준비합니다.
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 500

pad_X_train = pad_sequences(X_train, maxlen=max_len, padding = 'pre')
pad_X_test = pad_sequences(X_test, maxlen=max_len, padding = 'pre')

#### 모델 구성 및 Embedding Layer 가중치 전이

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
# 이 층은 모델의 제일 첫 번째 층으로만 사용할 수 있습니다.
# Flatten 층을 사용하기 위해 input_length를 전달합니다.
model.add(Embedding(input_dim = num_words, output_dim = embedding_dim, input_length = max_len))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics = ['acc'])

# embedding_matrix에 담겨있는 가중치를 Embedding Layer로 옮기고,
# 학습하지 않도록 동결시킵니다.
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

#### 학습시키기

In [ ]:
# 여기까지 Glove를 사용하여,
# 사전학습된 Embedding 층을 사용해보았습니다.
history = model.fit(pad_X_train, y_train, 
                    batch_size = 32, 
                    epochs = 30, 
                    validation_split = 0.2)